# Transformación de información

In [1]:
#Importar librerías
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from PIL import Image
import base64

In [2]:
# Parámetros de configuración - Modifica estos valores según tus necesidades
PDF_INPUT_PATH = "reglamento-de-estudiantes-universidad-javeriana.pdf"  # Ruta completa al archivo PDF
OUTPUT_FOLDER = "imagenes/"  # Carpeta donde se guardarán las imágenes
DPI = 300  # Resolución de las imágenes (puntos por pulgada)

# Importar las bibliotecas necesarias
import os
import matplotlib.pyplot as plt
from pdf2image import convert_from_path
from IPython.display import display, Image

def convert_pdf_to_images(pdf_path, output_folder, dpi=300):
    """
    Convierte un archivo PDF en imágenes JPG.
    
    Args:
        pdf_path (str): Ruta al archivo PDF.
        output_folder (str): Carpeta donde se guardarán las imágenes.
        dpi (int): Resolución de las imágenes (puntos por pulgada).
    
    Returns:
        list: Lista de rutas a las imágenes generadas.
    """
    # Verificar que el archivo PDF existe
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"El archivo PDF no existe en la ruta: {pdf_path}")
    
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Se ha creado la carpeta de salida: {output_folder}")
    
    # Convertir PDF a imágenes
    print(f"Convirtiendo PDF a imágenes con DPI={dpi}...")
    images = convert_from_path(pdf_path, dpi=dpi)
    
    # Guardar cada página como una imagen JPG
    image_paths = []
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f'pagina_{i+1}.jpg')
        image.save(image_path, 'JPEG')
        image_paths.append(image_path)
    
    print(f"Conversión completada. Se generaron {len(image_paths)} imágenes.")
    return image_paths

def main():
    """Función principal que ejecuta la conversión y muestra resultados."""
    try:
        # Mostrar información de los parámetros
        print("Parámetros de conversión:")
        print(f"- Archivo PDF: {PDF_INPUT_PATH}")
        print(f"- Carpeta de salida: {OUTPUT_FOLDER}")
        print(f"- Resolución (DPI): {DPI}")
        print("-" * 50)
        
        # Realizar la conversión
        image_paths = convert_pdf_to_images(PDF_INPUT_PATH, OUTPUT_FOLDER, DPI)
        
        # Mostrar la primera imagen como ejemplo
        if image_paths:
  
            
            # Mostrar información de todas las imágenes generadas
            print("\nImágenes generadas:")
            for i, path in enumerate(image_paths):
                print(f"- Página {i+1}: {path}")
        
    except Exception as e:
        print(f"Error durante la conversión: {str(e)}")

# Ejecutar la función principal
main()

Parámetros de conversión:
- Archivo PDF: reglamento-de-estudiantes-universidad-javeriana.pdf
- Carpeta de salida: imagenes/
- Resolución (DPI): 300
--------------------------------------------------
Convirtiendo PDF a imágenes con DPI=300...
Conversión completada. Se generaron 32 imágenes.

Imágenes generadas:
- Página 1: imagenes/pagina_1.jpg
- Página 2: imagenes/pagina_2.jpg
- Página 3: imagenes/pagina_3.jpg
- Página 4: imagenes/pagina_4.jpg
- Página 5: imagenes/pagina_5.jpg
- Página 6: imagenes/pagina_6.jpg
- Página 7: imagenes/pagina_7.jpg
- Página 8: imagenes/pagina_8.jpg
- Página 9: imagenes/pagina_9.jpg
- Página 10: imagenes/pagina_10.jpg
- Página 11: imagenes/pagina_11.jpg
- Página 12: imagenes/pagina_12.jpg
- Página 13: imagenes/pagina_13.jpg
- Página 14: imagenes/pagina_14.jpg
- Página 15: imagenes/pagina_15.jpg
- Página 16: imagenes/pagina_16.jpg
- Página 17: imagenes/pagina_17.jpg
- Página 18: imagenes/pagina_18.jpg
- Página 19: imagenes/pagina_19.jpg
- Página 20: imagenes/

In [4]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import HumanMessage
import base64
import os
import pandas as pd
from tqdm import tqdm  # Para mostrar una barra de progreso
import time 

def process_images_in_directory(directory_path, prompt):
    # Inicializar el modelo con Vertex AI
    model = ChatVertexAI(
        model_name="gemini-2.5-flash-preview-04-17",
        project="mestria-puj-s2",
        location="us-central1"
    )
    
    # Función para codificar la imagen
    def get_image_data(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    
    # Lista para almacenar los resultados
    results = []
    
    # Obtener la lista de archivos de imagen en el directorio
    image_files = [f for f in os.listdir(directory_path) 
                  if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    # Procesar cada imagen con una barra de progreso
    llamadas = 0

    for image_file in tqdm(image_files, desc="Procesando imágenes"):
        image_path = os.path.join(directory_path, image_file)
        
        try:
            image_message = HumanMessage(
                content=[
                    { "type": "text", "text": prompt },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{get_image_data(image_path)}"
                        }
                    }
                ]
            )
            
            response = model.invoke([image_message])
            
            results.append({
                "Nombre_archivo": image_file,
                "Resultado": response.content
            })
            print(f"Procesado: {image_file}")
            
            llamadas += 1
            # Cada 5 llamadas, esperar 60 segundos
            if llamadas % 5 == 0 and llamadas < len(image_files):
                print("Límite de 5 llamadas alcanzado. Esperando 60 segundos...")
                time.sleep(60)
            
        except Exception as e:
            print(f"Error al procesar {image_file}: {e}")
            results.append({
                "Nombre_archivo": image_file,
                "Resultado": f"ERROR: {e}"
            })
    
    df_results = pd.DataFrame(results)
    return df_results

# Prompt para el análisis
prompt="""
    Contexto:
    Eres un modelo de IA multimodal especializado en extraer texto de imágenes con máxima precisión. Recibirás como entrada la imagen de una página de un manual de estudiante. Esta información se utilizará para construir la base de conocimiento de un agente, por lo que no debes inventar ni parafrasear nada: solo extrae lo que ves.

    Instrucciones:
    1. Analiza la imagen y extrae TODO el contenido textual tal como aparece: títulos, subtítulos, párrafos, listas, tablas, pies de página, encabezados de columna, numeración, viñetas y cualquier otro texto.
    2. Conserva la ortografía, puntuación, saltos de línea, espacios y formato original (mayúsculas, negritas, cursivas, viñetas, numeración).
    3. No añadas comentarios, etiquetas, secciones ni explicaciones adicionales.
    4. No transformes ni reorganices el contenido: reproduce exactamente el orden y disposición del texto visible en la página.
    5. Si se presentan tablas, extrae cada tabla conservando filas y columnas, incluidos encabezados y líneas divisorias, usando un formato de texto con separadores simples (por ejemplo, “|”) y manteniendo los guiones o líneas de separación tal como aparecen.
    6. Mantén todas las enumeraciones y viñetas exactamente igual: conserva numeración, caracteres de viñeta y el espaciado original.

    Formato de salida:
    Devuelve un único string que contenga el texto íntegro extraído, respetando saltos de línea, separadores de tabla y el formato original, sin envoltorios ni secciones adicionales.
    """

# Ejecutar el procesamiento (puedes pasar la ruta como parámetro)
directory_path = "imagenes"  # Cambia esto a la ruta de tu directorio de imágenes
results_df = process_images_in_directory(directory_path, prompt)

Procesando imágenes:   3%|▎         | 1/32 [00:07<04:01,  7.78s/it]

Procesado: pagina_22.jpg


Procesando imágenes:   6%|▋         | 2/32 [00:16<04:12,  8.41s/it]

Procesado: pagina_8.jpg


Procesando imágenes:   9%|▉         | 3/32 [00:27<04:30,  9.34s/it]

...

Procesado: pagina_19.jpg
Procesado: pagina_6.jpg
Límite de 5 llamadas alcanzado. Esperando 60 segundos...


Procesando imágenes:  97%|█████████▋| 31/32 [11:00<00:24, 24.43s/it]

Procesado: pagina_15.jpg


Procesando imágenes: 100%|██████████| 32/32 [11:08<00:00, 20.90s/it]

Procesado: pagina_17.jpg


# Generación de embedding

In [5]:
# se inicializa vertex - toma credenciales default 

from google.cloud import aiplatform

# Reemplaza con tu proyecto y ubicación si es necesario
aiplatform.init(
    project="mestria-puj-s2",       
    location="us-central1"             
)

In [6]:
# carga del modelo de embeddings 

from vertexai.language_models import TextEmbeddingModel

# Cargar el modelo
model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")

In [7]:
import time
from tqdm import tqdm

embeddings = []
llamadas = 0
total_textos = len(results_df)

for text in tqdm(results_df['Resultado'], desc="Generando embeddings"):
    # Ejecutar llamada al servicio
    response = model.get_embeddings([text])
    embedding = response[0].values
    embeddings.append(embedding)
    
    llamadas += 1
    # Cada 5 llamadas, esperar 60 segundos (si aún hay más textos)
    if llamadas % 5 == 0 and llamadas < total_textos:
        print("Límite de 5 solicitudes alcanzado. Esperando 60 segundos...")
        time.sleep(60)

# Añadir columna al DataFrame
results_df["embedding"] = embeddings

Generando embeddings:  12%|█▎        | 4/32 [00:00<00:04,  5.75it/s]

Límite de 5 solicitudes alcanzado. Esperando 60 segundos...


Generando embeddings:  28%|██▊       | 9/32 [01:01<01:47,  4.65s/it]

Límite de 5 solicitudes alcanzado. Esperando 60 segundos...


Generando embeddings:  44%|████▍     | 14/32 [02:02<01:34,  5.23s/it]

...

Generando embeddings:  75%|███████▌  | 24/32 [04:03<00:42,  5.34s/it]

Límite de 5 solicitudes alcanzado. Esperando 60 segundos...


Generando embeddings:  91%|█████████ | 29/32 [05:04<00:16,  5.36s/it]

Límite de 5 solicitudes alcanzado. Esperando 60 segundos...


Generando embeddings: 100%|██████████| 32/32 [06:05<00:00, 11.42s/it]


In [8]:
import json

with open("embedding_manual.json", "w", encoding="utf-8") as f:
    for _, row in results_df.iterrows():
        # Línea de acción
        f.write(json.dumps({ "index": {} }) + "\n")

        # Documento en formato plano (sin _index ni _source)
        doc = {
            "embedding": row["embedding"],
            "text": row["Resultado"],
            "metadata": {
                "filename": row["Nombre_archivo"]
            }
        }
        f.write(json.dumps(doc) + "\n")